In [12]:
import typer
import click

from dataclasses import dataclass


from droughty import lookml_cli 
from droughty import dbt_test_cli
from droughty import dbml_cli
from droughty import cube_cli

app = typer.Typer()

app.add_typer(lookml_cli.app, name="lookml")
app.add_typer(dbt_test_cli.app, name="dbt")
app.add_typer(dbml_cli.app, name="dbml")
app.add_typer(cube_cli.app, name="cube")

##app.add_typer(config_cli.app)

#####

@dataclass
class Common:
    profile_dir: str

@app.callback()
def common(ctx: typer.Context,
           profile_dir: str = typer.Option(..., )):

    ##typer.echo(f"Hello {profile_dir}")

    """Common Entry Point"""
    ctx.obj = Common(profile_dir)

    return {profile_dir}

common_test = common()

print (common_test)



TypeError: common() missing 1 required positional argument: 'ctx'

In [22]:
class ClassA(object):
    def __init__(self):
        self.var1 = 1
        self.var2 = 2

    def methodA(self):
        self.var1 = self.var1 + self.var2
        return self.var1



class ClassB(ClassA):
    def __init__(self, class_a):
        self.var1 = class_a.var1
        self.var2 = class_a.var2

object1 = ClassA()
sum = object1.methodA()
object2 = ClassB(object1)
print (sum)

3


In [14]:
@dataclass
class Common:
    name: str

    @app.callback()
    def common(profile_dir: str):

        print ( "test_me " + profile_dir.name)

p1 = Common("john")

p1.common()

test_me john


In [15]:
class Person:
  def __init__(mysillyobject, name, age):
    mysillyobject.name = name
    mysillyobject.age = age

  def myfunc(abc):
    return "Hello my name is " + abc.name

p1 = Person("John", 36)

reference = p1.myfunc()

print (reference)

Hello my name is John


In [111]:
@dataclass
class Common:
    profile_dir: str

@app.callback()
def profile_callback(ctx: str = typer.Context,
                     ##ctx: str = typer.Context,
                     profile_dir: str = typer.Option(..., )):

    ##test_dir = (f"{profile_dir}")

    profile_dir = "test"

    return (profile_dir)

    """Common Entry Point"""
    
    ctx.obj = "test_me"

    return (ctx.obj)


cli_profile_path = profile_callback()

print (cli_profile_path)


test


In [83]:
class Example(object):
    itsProblem = "problem"


theExample = Example()
print(theExample.itsProblem)
print (Example.itsProblem)

problem
problem


In [ ]:
>>> class TestClass:
        def __init__(self):
            print("in init")
        def testFunc(self):
            print("in Test Func")


>>> testInstance = TestClass()
in init
>>> testInstance.testFunc()
in Test Func

In [43]:
print (reference)

Hello my name is John


In [3]:
path_source

'external_vars'

In [4]:
path

In [78]:
@dataclass
class Common:
    profile_dir: str

    @app.callback()
    def profile_callback(ctx: str = typer.Context,
                         profile_dir: str = typer.Option(..., )):

        typer.echo(f"Hello {profile_dir}")

        #test_dir = "/Users/lewischarlesbaker/.droughty/"

        test_dir = (f"{profile_dir}")

        """Common Entry Point"""
        ctx.obj = Common(profile_dir)

        return (test_dir)


cli_profile_path = Common.profile_callback()

Hello <typer.models.OptionInfo object at 0x111872970>


In [108]:
def profile_callback(ctx: str = typer.Context,
                     profile_dir: str = typer.Option(..., )):

    typer.echo(f"Hello {profile_dir}")

    #test_dir = "/Users/lewischarlesbaker/.droughty/"

    #test_dir = (f"{profile_dir}")

    """Common Entry Point"""
    test_dir = ctx.obj = Common(profile_dir)

    return (test_dir)

cli_profile_path = profile_callback()


Hello <typer.models.OptionInfo object at 0x111a75c40>


In [116]:
def profile_callback(##ctx: typer.Context,
                     profile_dir: str = typer.Option(..., )):

    typer.echo(f"Hello {profile_dir}")

    #test_dir = "/Users/lewischarlesbaker/.droughty/"

    test_dir = "{profile_dir}"

    """Common Entry Point"""
    ##ctx.obj = Common(profile_dir)

    return (test_dir)


print = profile_callback()


Hello <typer.models.OptionInfo object at 0x111a956a0>
